This notebook creates SQL databases that stores different tables from the MIMIC data.  
We first try SQL databse construction with the demo data from the MIMIC data.  
One SQLDdb is created. The DB will contain various tables from the MIMIC data repository.  

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [8]:
!pwd

/Users/michaeltang/Documents/Projects/biostats 823/BIOS823_Project/scripts


In [1]:
#read in the demo data via read_csv using pandas
admission = pd.read_csv('../_demo_data/ADMISSIONS.csv')
pres = pd.read_csv('../_demo_data/PRESCRIPTIONS.csv')
icu = pd.read_csv('../_demo_data/ICUSTAYS.csv')
pro_event = pd.read_csv('../_demo_data/PROCEDUREEVENTS_MV.csv')

NameError: name 'pd' is not defined

In [10]:
#connect to the previously created DB
con = sqlite3.connect('MIMIC3_demo.db')
#create cursor
cur = con.cursor()
#we write the admission table to the db
admission.to_sql('admissions', con, if_exists='replace', index = False)
#write the drug table to the db
pres.to_sql('prescriptions', con, if_exists='replace', index = False)
#write the icu table to the db
icu.to_sql('icustays', con, if_exists='replace', index = False)
#write the procedure table to the db
pro_event.to_sql('procedureevents', con, if_exists='replace', index = False)
#commit and close the db
con.commit()
con.close()

In [17]:
%load_ext sql
%sql sqlite:///../scripts/MIMIC3_demo.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @../scripts/MIMIC3_demo.db'

In [19]:
%%sql

SELECT * FROM admissions
limit 5

 * sqlite:///../scripts/MIMIC3_demo.db
   sqlite:///MIMIC3_demo.db
Done.


row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,None,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,None,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,None,None,HEPATITIS B,1,1
12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,None,UNKNOWN/NOT SPECIFIED,None,None,SEPSIS,1,1
12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,DIVORCED,WHITE,None,None,ALCOHOLIC HEPATITIS,1,1


In [21]:
#test the db by creating a sqlite3 query (create the con and read the query via pandas)
query = "SELECT * FROM admissions limit 5"
con = sqlite3.connect('MIMIC3_demo.db')
#leverage pd
pd.read_sql_query(query, con)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,None,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,None,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,None,None,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,None,UNKNOWN/NOT SPECIFIED,None,None,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,DIVORCED,WHITE,None,None,ALCOHOLIC HEPATITIS,1,1


In [22]:
con.close()